### Notebook used to generate list of congested branches: read requirements from input table, then generate desired output and copy to accessible location. 

In [1]:
import numpy as np
import pandas as pd

import os
import sys
sys.path.append("..")

from westernintnet.westernintnet import win_data

Loading sub
Loading bus2sub
Loading bus
Loading genbus
Loading branches
Loading resources
Loading net_generation
Done loading


In [2]:
wecc = win_data

* Read info file: table defines what input files go into the branch filter process and what outputfilenames should be
* Note that the File_Location can also be where files can be accessed by other programs, and may not be same as datadir


In [3]:
datadir = os.path.join('..','data/')
infofile_name = 'TransmissionCongestionInfo' + '.csv'
df = pd.read_csv(os.path.join(datadir, infofile_name), header=0, index_col = 'ID')

In [4]:
df.head()

,Source_Scenario_ID,Congestion_Model,Model_Region,Selected_Region,File_Location,Filename
ID,,,,,,
1,california2020Test01,Binomial,WECC,CA,L/RenewableEnergy/EnergyGridModeling/Data/West...,congestion_CA_only_california2020Test01_zscore...
2,california2030Test01,Binomial,WECC,CA,L/RenewableEnergy/EnergyGridModeling/Data/West...,congestion_CA_only_california2030Test01_zscore...


In [5]:
# Filter by zone; sample - here, filter by CA
# To do: generalize to any zone; needs a lookup table for region-loadzone designation.

CA_zones = [3,4,5,6,7]
CA_buses = wecc.bus.loc[wecc.bus['area'].isin(CA_zones)]
CA_branches = wecc.branches.loc[(wecc.branches['tbus'].isin(CA_buses['busID'])) 
                              & (wecc.branches['fbus'].isin(CA_buses['busID']))]

fix_list = df.Source_Scenario_ID.tolist()

In [7]:
congestion_dfs = {}
CA_fix_congestion = {}

# To do: Currently, i and s are interchangeable indexes; 
# eventually, s may be used as source scenario for other outputs

for i in df.index:
    s = df.loc[i]['Source_Scenario_ID']
    congestion_source = datadir + 'congestion_' + s +'.csv'
    congestion_dfs[s] = pd.read_csv(congestion_source)  
    CA_fix_congestion[s] = congestion_dfs[s].loc[(congestion_dfs[s]['line'].isin(CA_branches.index)) 
                                                 & (congestion_dfs[s]['zscore']>=1)]
    out_filename1 = datadir + df.loc[i]['Filename']  
    
    # group-accessible folder
    out_filename2 = df.loc[i]['File_Location'] + df.loc[i]['Filename']  

    if not os.path.exists(out_filename1):
        CA_fix_congestion[s].to_csv(out_filename1)
    
    #To do: 
    # Copy to group accessible server folder
    # Put in a check for whether the server L folder is mounted
    #if not os.path.exists(out_filename2):
    #    CA_fix_congestion[s].to_csv(out_filename2)
                
    print(s, len(CA_fix_congestion[s]))

california2020Test01 51
california2030Test01 129
